In [1]:
# Module Importations
import datetime
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
# Project Module Importations
from data_analytics.src.data import cosmos_client_manager
from data_analytics.src.data import load_data
from data_analytics.src.data import test_unit
from data_analytics.src.features import data_munging

In [3]:
# Load data from local store
df_new = load_data.load_dash_dataframe()

Loaded Dash dataframe ...
<class 'pandas.core.frame.DataFrame'>
Int64Index: 22315 entries, 0 to 49092
Data columns (total 16 columns):
id             22315 non-null object
docType        22315 non-null object
busType        22315 non-null object
proType        22315 non-null object
timeId         22315 non-null object
quantity       22315 non-null float64
curveType      22315 non-null object
resolution     22315 non-null object
setDate        22315 non-null object
setPeriod      22315 non-null float64
powType        22315 non-null object
actFlag        22315 non-null object
docId          22315 non-null object
docRevNum      22315 non-null object
_ts            22315 non-null int64
setDatetime    22315 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(12)
memory usage: 2.9+ MB
None


In [4]:
aggregated_generation = {
        "Solar": 0,
        "Wind Onshore": 0
        }

for key in aggregated_generation:
    print(key)
    df_generation = df_new[df_new['powType'] == key]

Solar
Wind Onshore


In [5]:
df_test = test_unit.return_aggregate_df(df_new)

NameError: name 'df_new' is not defined